# Hello world using Flask

In [27]:
import os
hello_world_script_file = os.path.join( os.path.pardir, 'src', 'models','hello_world_api.py' )

In [28]:
%%writefile $hello_world_script_file

from flask import Flask, request

app = Flask(__name__)

@app.route('/api', methods=['POST'])
def say_hello():
    data = request.get_json(force=True)
    name = data['name']
    return 'Hello {0}'.format(name)

if __name__ == '__main__':
    app.run(port=10001, debug=True)

Overwriting ..\src\models\hello_world_api.py


In [21]:
import requests
import json

In [22]:
url = 'http://127.0.0.1:10001/api'
data = json.dumps({'name': 'Oscal'})
r = requests.post(url, data)

In [23]:
print(r.text)

Hello Oscal


## Building ML API

In [29]:
hello_world_script_file = os.path.join( os.path.pardir, 'src', 'models','machine_learning_api.py' )

In [47]:
%%writefile $hello_world_script_file
from flask import Flask, request
import pandas as pd
import numpy as np
import json
import pickle
import os

model_path = os.path.join('C:\\Users\\Kiosk\\Documents\\titanic\\', 'models')
model_filepath = os.path.join(model_path, 'lr_model.pkl')

model = pickle.load(open(model_filepath, 'rb'))

columns = [ 'Age', 'Fare', 'FamilySize', 'isMother', 'isMale', 'Deck_A',
       'Deck_B', 'Deck_C', 'Deck_D', 'Deck_E', 'Deck_F', 'Deck_G', 'Deck_Z',
       'Pclass_1', 'Pclass_2', 'Pclass_3', 'Title_Lady', 'Title_Master',
       'Title_Miss', 'Title_Mr', 'Title_Mrs', 'Title_Officer', 'Title_Sir',
       'Fare_bins_Very_Low', 'Fare_bins_Low', 'Fare_bins_High',
       'Fare_bins_Very_High', 'Embarked_C', 'Embarked_Q', 'Embarked_S',
       'AgeState_Adult', 'AgeState_Child']

app = Flask(__name__)

@app.route('/api', methods=['POST'])
def make_predictions():
    data = json.dumps(request.get_json(force=True))
    df = pd.read_json(data)
    
    passenger_ids = df['PassengerId'].ravel()
    actuals = df['Survived'].ravel()
    
    X = df[columns].as_matrix().astype('float')
    
    predictions = model.predict(X)
    
    df_response = pd.DataFrame({'PassengerId': passenger_ids, 'Predictions': predictions, 'Actuals': actuals})
    
    return df_response.to_json()

if __name__ == '__main__':
    app.run(port=10001, debug=True)

Overwriting ..\src\models\machine_learning_api.py


In [48]:
import os
import pandas as pd

In [50]:
processed_data_path =os.path.join(os.path.pardir, 'data', 'processed' )
train_file_path = os.path.join(processed_data_path, 'train.csv')
train_df = pd.read_csv(train_file_path)

In [51]:
survived_passengers=train_df[train_df.Survived == 1][:5]

In [52]:
survived_passengers

,PassengerId,Survived,Age,Fare,FamilySize,isMother,isMale,Deck_A,Deck_B,Deck_C,...,Title_Sir,Fare_bins_Very_Low,Fare_bins_Low,Fare_bins_High,Fare_bins_Very_High,Embarked_C,Embarked_Q,Embarked_S,AgeState_Adult,AgeState_Child
1,2,1,38.0,71.2833,2,0,0,0,0,1,...,0,0,0,0,1,1,0,0,1,0
2,3,1,26.0,7.9250,1,0,0,0,0,0,...,0,0,1,0,0,0,0,1,1,0
3,4,1,35.0,53.1000,2,0,0,0,0,1,...,0,0,0,0,1,0,0,1,1,0
8,9,1,27.0,11.1333,3,1,0,0,0,0,...,0,0,1,0,0,0,0,1,1,0
9,10,1,14.0,30.0708,2,0,0,0,0,0,...,0,0,0,1,0,1,0,0,0,1


In [53]:
import requests
def make_request(data):
    url = 'http://127.0.0.1:10001/api'
    r=requests.post(url,data)
    return r.json()

In [54]:
make_request(survived_passengers.to_json())

{'PassengerId': {'0': 2, '1': 3, '2': 4, '3': 9, '4': 10},
 'Predictions': {'0': 1, '1': 1, '2': 1, '3': 1, '4': 1},
 'Actuals': {'0': 1, '1': 1, '2': 1, '3': 1, '4': 1}}

In [57]:
result = make_request(train_df.to_json())
df_res = pd.read_json(json.dumps(result))
df_res.head()

,PassengerId,Predictions,Actuals
0,1,0,0
1,2,1,1
10,108,0,1
100,190,0,0
101,20,1,1


In [59]:
import numpy as np
np.mean(df_res.Actuals == df_res.Predictions)

0.8372615039281706